# Objective
Identify a long term conservative risk investing strategy
- **Part 1** : Visualize historic growth of Equities - ETFs / Stocks
- **Part 2** : Evaluate todays returns using historic data if you had invested x amount in the stock / ETF

In [35]:
# load libraries
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

In [12]:
# giving the start and end dates
startDate = '2010-01-01'
endDate = '2024-06-01'

# setting the ticker value
tickers = ['^IXIC','^GSPC','VOO','AAPL','QQQ','VTI','VBR']
names = ['NASDAQ','SnP500','VOO_Van_Snp500','Apple','QQQ_Nasdaq-100','VTI_Van_Total_Stock','VBR_Van_Small_Cap']
stocks_df = pd.DataFrame()
for ticker in tickers:
    # downloading the data of the ticker value between
    # the start and end dates
    stocks_df[ticker] = yf.download(ticker, startDate, endDate)['Adj Close']

stocks_df.columns = names
# printing the last 5 rows of the data
print(stocks_df.tail())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                  NASDAQ       SnP500  VOO_Van_Snp500       Apple  \
Date                                                                
2024-05-13  16388.240234  5221.419922      478.769989  186.279999   
2024-05-14  16511.179688  5246.680176      481.040009  187.429993   
2024-05-15  16742.390625  5308.149902      486.899994  189.720001   
2024-05-16  16698.320312  5297.100098      485.970001  189.839996   
2024-05-17  16685.970703  5303.270020      486.690002  189.869995   

            QQQ_Nasdaq-100  VTI_Van_Total_Stock  VBR_Van_Small_Cap  
Date                                                                
2024-05-13      443.079987           258.190002         188.490005  
2024-05-14      445.929993           259.450012         190.169998  
2024-05-15      452.899994           262.640015         191.110001  
2024-05-16      451.980011           261.929993         189.970001  
2024-05-17      451.760010           262.299988         190.080002  


## Part 1

In [13]:
# Plot raw data
px.line(data_frame = stocks_df, y = names[2:])

In [15]:
# Check missing values
stocks_df.isnull().sum()

NASDAQ                   0
SnP500                   0
VOO_Van_Snp500         172
Apple                    0
QQQ_Nasdaq-100           0
VTI_Van_Total_Stock      0
VBR_Van_Small_Cap        0
dtype: int64

In [19]:
# Create a heatmap of the missing values
fig = px.imshow(stocks_df.isna())

# Update the layout
fig.update_layout(title='Missing Values Heatmap')

# Show the plot
fig.show()

Notes:
- VOO seems to be newer than the other Equities. Hence, null values at the start of the series
- Different equites have different start prices making it hard to infer the growth 

In [23]:
# Drop initial missing values
stocks_df.dropna(inplace=True)
# Normalize the stock data by making the series start price as $1
norm_stocks_df = stocks_df / stocks_df.iloc[0,:]

In [29]:
# Plot raw data
fig = px.line(data_frame = norm_stocks_df, y = names[1:],
              title="Normalized Stock Data")
fig.show()

Notes:
- Seems like Apple outperformed other indexes. It is risky to invest everything into Apple. Especially if you are not managing your portfolio actively. Observing QQQ - Nasdaq 100, One can conclude bigger companies grow faster.
- Unusual growth post pandemic. To get generalized view, we'd have to check returns excluding the anomaly as well

### Calculating Stock Performance
There are a few metrics to calculate the performance of a stock:

**Annualized Returns** = average yearly gain in stock price over time, the higher the better

[**Annualized Volatility**](https://quant.stackexchange.com/questions/51677/why-multiply-stock-returns-with-sqrt252)  = average yearly fluctuations of stock price over time, the lower the better

In [38]:
def stock_performance(df):
    print('*' * 100)
    print('Annualized Returns')
    print('*' * 100)
    annualized_returns = df.pct_change().mean().apply(lambda x: x*252)
    # calculate the mean of the Daily Returns Ratio and multiply it by 252 (roughly the number of trading days in a year) 
    # to get the Annualized Returns
    print(annualized_returns.sort_values(ascending=False))
    print('*' * 100)
    print('Annualized Volatility')
    print('*' * 100)
    annualized_volatility = df.pct_change().std().apply(lambda x: x*np.sqrt(252))
    print(annualized_volatility.sort_values(ascending=False))

In [39]:
stock_performance(stocks_df)

****************************************************************************************************
Annualized Returns
****************************************************************************************************
Apple                  0.271521
QQQ_Nasdaq-100         0.196650
NASDAQ                 0.167562
VOO_Van_Snp500         0.148316
VTI_Van_Total_Stock    0.146100
VBR_Van_Small_Cap      0.131407
SnP500                 0.129723
dtype: float64
****************************************************************************************************
Annualized Volatility
****************************************************************************************************
Apple                  0.280050
VBR_Van_Small_Cap      0.208387
QQQ_Nasdaq-100         0.204450
NASDAQ                 0.202145
VTI_Van_Total_Stock    0.174624
SnP500                 0.172348
VOO_Van_Snp500         0.171784
dtype: float64


In [42]:
# Exclude data post 2020
stock_performance(stocks_df.loc[:'2020-01-01',])

****************************************************************************************************
Annualized Returns
****************************************************************************************************
Apple                  0.268647
QQQ_Nasdaq-100         0.188740
NASDAQ                 0.163346
VOO_Van_Snp500         0.145564
VTI_Van_Total_Stock    0.144952
VBR_Van_Small_Cap      0.130681
SnP500                 0.125734
dtype: float64
****************************************************************************************************
Annualized Volatility
****************************************************************************************************
Apple                  0.254528
QQQ_Nasdaq-100         0.169031
VBR_Van_Small_Cap      0.168156
NASDAQ                 0.166363
VTI_Van_Total_Stock    0.144982
SnP500                 0.142922
VOO_Van_Snp500         0.142202
dtype: float64


## Part 2
Investing strategy 1: investing x amount on each stock every week

## References
- [Differences between S&P, Dow and Nasdaq](https://www.cnbc.com/2023/07/21/difference-between-the-sp-the-dow-and-the-nasdaq.html)
- [Analyzing the Stock Market](https://medium.com/alpha-beta-blog/analyzing-the-stock-market-with-python-part-i-39f14670771b)
- 